In [3]:
import pandas as pd

In [4]:
usr_eng = pd.read_csv("takehome_user_engagement.csv")
usr_eng.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [22]:
users = pd.read_csv("takehome_users.csv",encoding="ISO-8859-1")


In [12]:
visits = usr_eng["user_id"].value_counts()
adopted = visits[visits >= 3]
adopted.head()

3623    606
906     600
1811    593
7590    590
8068    585
Name: user_id, dtype: int64

In [13]:
for index, val in adopted.iteritems(): 
    users.loc[users['object_id'] == index, 'adopted_user'] = 1 #index is used to connect with users df 'object_id' column
    
#Check that transformation went through with confirmed adopted user
users.loc[users['object_id'] == 3623]

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,adopted_user
3622,3623,6/25/2012 14:34,Melo Vitória,VitoriaAlvesMelo@jourrapide.com,ORG_INVITE,1.401201e+09,0,0,364,3623.0,1.0


In [15]:
#Drop all NaN values 
users = users.fillna(0)

In [16]:
 # drop irrelevant columns, leaving only "object_id", "opted_in_to_mailing_list", "enabled_for_marketing_drip" and 'adopted_user'
users.drop(['creation_time', 'name', 'email', 'creation_source', 'last_session_creation_time', 'org_id', 'invited_by_user_id'], axis=1, inplace=True)

In [17]:
# Training Set
train = users[:9600] #Training Set

In [18]:
# Test data set
test = users[9600:]

In [19]:
#Setting up Training and Test  dataset

x_train = train.drop('adopted_user', axis = 1)
y_train= train['adopted_user']
x_estx = test.drop('adopted_user', axis = 1)
y_test = test['adopted_user']

In [23]:
#implementing regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

lm = LogisticRegression(class_weight = 'balanced').fit(x_train,y_train)

In [25]:
lm_train_predictions =  lm.predict(x_train)
report = classification_report(y_train, lm_train_predictions)
print(report)

             precision    recall  f1-score   support

        0.0       0.82      0.71      0.76      7819
        1.0       0.19      0.30      0.24      1781

avg / total       0.70      0.64      0.66      9600



In [27]:
#Building Decision Tree (DT) model

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

param_grid = [{'max_depth': [1, 2, 3, 4]}]
tree_class = DecisionTreeClassifier()
grid_search = GridSearchCV(tree_class, param_grid, cv = 3, scoring = 'neg_mean_squared_error')
grid_search.fit(x_train,y_train)

#Getting best estimator directly. Inputting results in refit Decision Tree model.
grid_search.best_estimator_

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [29]:
#Training DT Model with optimized parameters

tree_class = DecisionTreeClassifier(class_weight = 'balanced', max_depth = 1)
tree_class.fit(x_train,y_train)

DecisionTreeClassifier(class_weight='balanced', criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [32]:
#Predicting on Training dataset
tree_class_train_predictions = tree_class.predict(x_train)
tree_class_train_predictions

array([ 0.,  0.,  0., ...,  1.,  1.,  1.])

In [ ]:
# Decision Tree model seems to have much better success rate than other models.